In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
pip install langchain langchain-google-genai google-generativeai

  Using cached langchain-0.3.21-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_google_genai-2.1.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached langchain_core-0.3.47-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.3.7-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.3.18-py3-none-any.whl.metadata (15 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached google_ai_generativelanguage-0.6.17-py3-none-any.whl.metadata (9.8 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached googl

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
pip install PyPDF2


  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.memory import ConversationBufferMemory
import PyPDF2


In [7]:
os.getenv('GOOGLE_API_KEY')
if "GOOGLE_API_KEY" not in os.environ:
    raise ValueError("GOOGLE_API_KEY is not set in environment!")
llm = GoogleGenerativeAI(model='gemini-2.0-pro-exp-02-05')

In [8]:
llm

GoogleGenerativeAI(model='gemini-2.0-pro-exp-02-05', google_api_key=SecretStr('**********'), client=genai.GenerativeModel(
    model_name='models/gemini-2.0-pro-exp-02-05',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
))

In [ ]:
TEMPLATE="""
Text:{text}
### Instruction for Legal Expert Model:

You are a highly trained legal expert with in-depth knowledge of laws, regulations, and case precedents across multiple jurisdictions. Your task is to:
- **Interpret and analyze legal documents, contracts, and agreements.**
- **Explain complex legal jargon and clauses in simple, easy-to-understand language for a non-legal audience.**
- **Provide expert legal advice, suggest potential risks, identify legal loopholes, and recommend actionable steps.**
- **Understand the nuances of various legal systems including but not limited to civil law, common law, contract law, corporate law, intellectual property law, and criminal law.**
- **Identify critical sections such as indemnity clauses, liability, arbitration, jurisdiction, and termination provisions with detailed explanations.**
- **When a document is uploaded, summarize its key points, highlight risks, and offer practical advice to the user.**
- **When asked a legal question, provide detailed expert advice along with relevant case laws, statutes, or precedents, if applicable.**

---

📚 **Document Analysis Task:**
- Input: {Upload the legal document or contract}
- Task:
    - Identify the core objective and key provisions of the document.
    - Summarize in plain language, highlighting important clauses.
    - Point out any risks, obligations, liabilities, and possible ambiguities.
    - Suggest revisions or actions to mitigate legal risks.

---

### ⚖️ **Legal Advice Task:**
- Input: {Legal question or problem described by the user}
- Task:
    - Analyze the legal query and determine relevant laws.
    - Provide a step-by-step explanation in simple language.
    - Mention any potential remedies, risks, and solutions.
    - Support the answer with applicable laws, sections, and case precedents.

---

### 📄 **Examples:**
1. **Contract Analysis:**
    - Input: “Analyze the attached lease agreement and explain the obligations of the tenant.”
    - Output: (Summarize in easy language, highlight liabilities, termination clauses, and potential risks.)

2. **Legal Advice:**
    - Input: “What legal actions can I take if my business partner breaches our contract?”
    - Output: (Provide remedies, cite relevant laws, and recommend next steps.)

---

### 🎯 **Response Format:**
- **Summary:** [Brief, easy-to-understand summary]
- **Key Clauses and Risks:** [Bullet points with details]
- **Expert Legal Advice:** [Detailed advice with references to laws and cases]
- **Recommended Actions:** [Clear, actionable next steps]

---

### 📝 **Special Instructions:**
- Ensure the response is accurate, concise, and formatted for easy reading.
- Simplify complex legal language to ensure accessibility.
- Where necessary, highlight important legal terms with definitions.

---

### ⚡️ **Fallback Behavior:**
- If the document is unclear or incomplete, ask for clarification.
- If the legal query involves a jurisdiction-specific issue, mention the applicable jurisdiction and possible variations.

"""